In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [2]:
ds = pd.read_csv('AW_BikeBuyer.csv')
ds2 = pd.read_csv('AdvWorksCusts.csv')
total=pd.merge(ds,ds2)
print(total.shape)
total.head()

(16749, 24)


,CustomerID,BikeBuyer,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,0,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,...,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,1,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,...,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,0,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,...,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,0,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,...,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,1,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,...,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771


In [3]:
labels = np.array(total['BikeBuyer'])

In [4]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['CountryRegionName',  'Gender', 
                       'MaritalStatus', 'Education']

Features = encode_string(total['Occupation'])
for col in categorical_columns:
    temp = encode_string(total[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :]) 

(16749, 20)
[[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0.]]


In [5]:
Features = np.concatenate([Features, np.array(total[['NumberCarsOwned', 
                            'NumberChildrenAtHome', 'YearlyIncome']])], axis = 1)
print(Features.shape)
print(Features[:2, :])

(16749, 23)
[[0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.37947e+05]
 [0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 0.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 1.00000e+00 3.00000e+00 1.01141e+05]]


In [6]:
total=total.drop(columns=['AddressLine2','Title','MiddleName','Suffix'])
total.head()

,CustomerID,BikeBuyer,FirstName,LastName,AddressLine1,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,0,Jon,Yang,3761 N. 14th St,Rockhampton,Queensland,Australia,4700,1 (11) 500 555-0162,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,1,Eugene,Huang,2243 W St.,Seaford,Victoria,Australia,3198,1 (11) 500 555-0110,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,0,Ruben,Torres,5844 Linden Land,Hobart,Tasmania,Australia,7001,1 (11) 500 555-0184,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,0,Christy,Zhu,1825 Village Pl.,North Ryde,New South Wales,Australia,2113,1 (11) 500 555-0162,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,1,Elizabeth,Johnson,7553 Harness Circle,Wollongong,New South Wales,Australia,2500,1 (11) 500 555-0131,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771


In [7]:

print(total.shape)

(16749, 20)


In [ ]:
test = pd.read_csv('AW_test.csv')
categorical_columns = ['CountryRegionName',  'Gender', 
                       'MaritalStatus', 'Education']

Features1 = encode_string(test['Occupation'])
for col in categorical_columns:
    temp = encode_string(total[col])
    Features1 = np.concatenate([Features1, temp], axis = 1)

print(Features1.shape)
print(Features1[:2, :]) 

Features1 = np.concatenate([Features1, np.array(total[['NumberCarsOwned', 
                            'NumberChildrenAtHome', 'YearlyIncome']])], axis = 1)
print(Features1.shape)
print(Features1[:2, :])

test=test.drop(columns=['AddressLine2','Title','MiddleName','Suffix'])
test.head()

print(test.shape)

In [ ]:
print(test.shape)
test=test.drop(columns=['AddressLine2','Title','MiddleName','Suffix'])
print(test.shape)

In [8]:
nr.seed(9988)

indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 300)

X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [9]:
scaler = preprocessing.StandardScaler().fit(X_train[:,22:])
X_train[:,22:] = scaler.transform(X_train[:,22:])
X_test[:,22:] = scaler.transform(X_test[:,22:])
X_train[:2,]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        , -0.38991748],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         2.        ,  4.        , -1.41059416]])

In [10]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
print(logistic_mod.intercept_)
print(logistic_mod.coef_)

[-0.62389367]
[[ 0.04628209 -1.0437784   0.32889819 -0.09626012  0.14096457 -0.11852492
  -0.01205831 -0.10061864 -0.15155881 -0.13522169 -0.1059113  -0.62022594
  -0.00366773 -1.10389599  0.48000233  0.29713568  0.04750006 -0.26544371
  -0.13352944 -0.56955626 -0.07620182  0.76758595  0.61664078]]


In [12]:
probabilities = logistic_mod.predict_proba(X_test)
print(probabilities[:15,:])

[[0.49547091 0.50452909]
 [0.83383428 0.16616572]
 [0.68659566 0.31340434]
 [0.92660585 0.07339415]
 [0.72860894 0.27139106]
 [0.85551332 0.14448668]
 [0.91285805 0.08714195]
 [0.96318176 0.03681824]
 [0.7278778  0.2721222 ]
 [0.94287991 0.05712009]
 [0.83993368 0.16006632]
 [0.87805267 0.12194733]
 [0.86975067 0.13024933]
 [0.9254621  0.0745379 ]
 [0.84662288 0.15337712]]


In [13]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(np.array(scores[:15]))
print(y_test[:15])

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 1 0 0 1 0]


In [14]:
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])


    
print_metrics(y_test, scores) 

                 Confusion matrix
                 Score positive    Score negative
Actual positive       199                14
Actual negative        36                51

Accuracy  0.83
 
           Positive      Negative
Num case      213            87
Precision    0.85          0.78
Recall       0.93          0.59
F1           0.89          0.67
